In [1]:
# pip install torch transformers tqdm

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from tqdm import tqdm
import os

In [3]:
# def verify_cuda_setup():
#     """
#     Verify CUDA setup and provide detailed information
#     """
#     print("\n=== CUDA Setup Verification ===")
    
#     # Check PyTorch installation
#     print(f"PyTorch Version: {torch.__version__}")
#     print(f"CUDA available: {torch.cuda.is_available()}")
    
#     if torch.cuda.is_available():
#         # Get CUDA version
#         cuda_version = torch.version.cuda
#         print(f"CUDA Version: {cuda_version}")
        
#         # GPU Device information
#         current_device = torch.cuda.current_device()
#         print(f"Current CUDA device index: {current_device}")
#         print(f"GPU Device: {torch.cuda.get_device_name(current_device)}")
        
#         # Memory information
#         print("\nGPU Memory Information:")
#         print(f"Total GPU Memory: {torch.cuda.get_device_properties(current_device).total_memory / 1024**3:.2f} GB")
#         print(f"Memory Allocated: {torch.cuda.memory_allocated(current_device) / 1024**3:.2f} GB")
#         print(f"Memory Cached: {torch.cuda.memory_reserved(current_device) / 1024**3:.2f} GB")
        
#         # CUDA architecture
#         print(f"\nCUDA Architecture:")
#         print(f"Device Capability: {torch.cuda.get_device_capability(current_device)}")
        
#         # Test CUDA computation
#         print("\nTesting CUDA computation...")
#         try:
#             test_tensor = torch.tensor([1.0, 2.0, 3.0], device='cuda')
#             result = test_tensor.sum()
#             print("CUDA computation test: Successful")
#         except Exception as e:
#             print(f"CUDA computation test failed: {e}")
            
#         return torch.device('cuda')
#     else:
#         print("\nWARNING: CUDA is not available. Check your PyTorch installation and NVIDIA drivers.")
#         print("\nDebug Information:")
#         print(f"PyTorch CUDA arch list: {os.environ.get('TORCH_CUDA_ARCH_LIST', 'Not Set')}")
#         print(f"CUDA_HOME: {os.environ.get('CUDA_HOME', 'Not Set')}")
#         print(f"CUDA_PATH: {os.environ.get('CUDA_PATH', 'Not Set')}")
#         return torch.device('cpu')

In [4]:
def verify_cuda_setup():
    """
    Verify CUDA setup and provide detailed information
    """
    print("\n=== CUDA Setup Verification ===")
    
    # Check PyTorch installation
    print(f"PyTorch Version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        # Get CUDA version
        cuda_version = torch.version.cuda
        print(f"CUDA Version: {cuda_version}")
        
        # GPU Device information
        current_device = torch.cuda.current_device()
        print(f"Current CUDA device index: {current_device}")
        print(f"GPU Device: {torch.cuda.get_device_name(current_device)}")
        
        # Memory information
        print("\nGPU Memory Information:")
        print(f"Total GPU Memory: {torch.cuda.get_device_properties(current_device).total_memory / 1024**3:.2f} GB")
        print(f"Memory Allocated: {torch.cuda.memory_allocated(current_device) / 1024**3:.2f} GB")
        print(f"Memory Cached: {torch.cuda.memory_reserved(current_device) / 1024**3:.2f} GB")
        
        # CUDA architecture
        print(f"\nCUDA Architecture:")
        print(f"Device Capability: {torch.cuda.get_device_capability(current_device)}")
        
        # Test CUDA computation
        print("\nTesting CUDA computation...")
        try:
            test_tensor = torch.tensor([1.0, 2.0, 3.0], device='cuda')
            result = test_tensor.sum()
            print("CUDA computation test: Successful")
        except Exception as e:
            print(f"CUDA computation test failed: {e}")
            
        return torch.device('cuda')
    else:
        print("\nWARNING: CUDA is not available. Check your PyTorch installation and NVIDIA drivers.")
        print("\nDebug Information:")
        print(f"PyTorch CUDA arch list: {os.environ.get('TORCH_CUDA_ARCH_LIST', 'Not Set')}")
        print(f"CUDA_HOME: {os.environ.get('CUDA_HOME', 'Not Set')}")
        print(f"CUDA_PATH: {os.environ.get('CUDA_PATH', 'Not Set')}")
        return torch.device('cpu')

In [5]:
verify_cuda_setup()


=== CUDA Setup Verification ===
PyTorch Version: 2.5.1+cu121
CUDA available: True
CUDA Version: 12.1
Current CUDA device index: 0
GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU

GPU Memory Information:
Total GPU Memory: 4.00 GB
Memory Allocated: 0.00 GB
Memory Cached: 0.00 GB

CUDA Architecture:
Device Capability: (8, 6)

Testing CUDA computation...
CUDA computation test: Successful


device(type='cuda')

In [6]:
import nltk

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
def load_and_preprocess_data():
    """Load and preprocess the Twitter dataset"""
    # column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
    data = pd.read_csv("IMDB_Dataset.csv", encoding='ISO-8859-1')
    # data['target'] = data['category'].replace(-1, 2)
    return data

In [8]:
data = load_and_preprocess_data()
# data = data.sample(n=30000, random_state=42)
data.head()
data.shape

(50000, 2)

In [9]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
data['target'] = data['sentiment'].replace('positive', 1).replace('negative', 0)

C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\3214355603.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['target'] = data['sentiment'].replace('positive', 1).replace('negative', 0)


In [11]:
data = data.dropna()

In [12]:
data.isnull().sum()

review       0
sentiment    0
target       0
dtype: int64

In [13]:
# checking the distribution of target col

data['target'].value_counts()

target
1    25000
0    25000
Name: count, dtype: int64

In [14]:
data.head()

,review,sentiment,target
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [15]:
import re

In [16]:
def advanced_text_preprocessing(content):
    # Check if content is a string, if not convert to string or return empty string
    if not isinstance(content, str):
        try:
            content = str(content)
        except:
            return ''

    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Convert to lowercase and remove special characters
    lemmatized_content = re.sub(r'http\S+|www\S+|https\S+', '', content, flags=re.MULTILINE)  # Remove URLs
    lemmatized_content = re.sub(r'@\w+|\#', '', lemmatized_content)  # Remove mentions and hashtag symbols
    lemmatized_content = re.sub(r'[^a-zA-Z\s]', '', lemmatized_content)  # Remove numbers and punctuation
    lemmatized_content = lemmatized_content.lower()

    # Tokenization
    words = word_tokenize(lemmatized_content)
    
    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    # Add custom stopwords that might not be useful for sentiment
    custom_stopwords = {'rt', 'via'}
    stop_words.update(custom_stopwords)
    processed_words = [
        lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 2
    ]
    lemmatized_content = ' '.join(processed_words)
    return lemmatized_content

In [17]:
 # Apply advanced text preprocessing
    
print("Applying text preprocessing...")
    
data['lemmatized_content'] = data['review'].apply(advanced_text_preprocessing)

Applying text preprocessing...


In [18]:
data.head()

,review,sentiment,target,lemmatized_content
0,One of the other reviewers has mentioned that ...,positive,1,one reviewer mentioned watching episode youll ...
1,A wonderful little production. <br /><br />The...,positive,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,0,basically there family little boy jake think t...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,petter matteis love time money visually stunni...


In [19]:
# Check for non-string values
print(data['lemmatized_content'].dtype)
print(data['lemmatized_content'].isnull().sum())
print(data[data['lemmatized_content'].apply(lambda x: not isinstance(x, str))].head())

object
0
Empty DataFrame
Columns: [review, sentiment, target, lemmatized_content]
Index: []


In [20]:
class TwitterDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=128):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [21]:
# def train_bert_model(data, epochs=3, batch_size=32, learning_rate=2e-5):
#     """
#     Train a BERT model for sentiment analysis
    
#     Parameters:
#     data: pandas DataFrame with 'lemmatized_content' and 'target' columns
#     epochs: number of training epochs
#     batch_size: batch size for training
#     learning_rate: learning rate for optimization
#     """
#     # Set device
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print(f"Using device: {device}")

#     # Initialize tokenizer and model
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     model = BertForSequenceClassification.from_pretrained(
#         'bert-base-uncased',
#         num_labels=2
#     ).to(device)

#     # Prepare data
#     X = data['lemmatized_content'].values
#     y = data['target'].values
    
#     X_train, X_val, y_train, y_val = train_test_split(
#         X, y, test_size=0.2, stratify=y, random_state=42
#     )

#     # Create datasets
#     train_dataset = TwitterDataset(X_train, y_train, tokenizer)
#     val_dataset = TwitterDataset(X_val, y_val, tokenizer)

#     # Create data loaders
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=batch_size)

#     # Initialize optimizer
#     optimizer = AdamW(model.parameters(), lr=learning_rate)

#     # Training loop
#     best_accuracy = 0
#     for epoch in range(epochs):
#         print(f'\nEpoch {epoch + 1}/{epochs}')
        
#         # Training phase
#         model.train()
#         train_losses = []
#         train_preds = []
#         train_true = []

#         for batch in tqdm(train_loader, desc='Training'):
#             optimizer.zero_grad()
            
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             targets = batch['targets'].to(device)

#             outputs = model(
#                 input_ids=input_ids,
#                 attention_mask=attention_mask,
#                 labels=targets
#             )

#             loss = outputs.loss
#             loss.backward()
#             optimizer.step()

#             train_losses.append(loss.item())
            
#             preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
#             train_preds.extend(preds)
#             train_true.extend(targets.cpu().numpy())

#         train_accuracy = accuracy_score(train_true, train_preds)
        
#         # Validation phase
#         model.eval()
#         val_losses = []
#         val_preds = []
#         val_true = []

#         with torch.no_grad():
#             for batch in tqdm(val_loader, desc='Validation'):
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 targets = batch['targets'].to(device)

#                 outputs = model(
#                     input_ids=input_ids,
#                     attention_mask=attention_mask,
#                     labels=targets
#                 )

#                 val_losses.append(outputs.loss.item())
                
#                 preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
#                 val_preds.extend(preds)
#                 val_true.extend(targets.cpu().numpy())

#         val_accuracy = accuracy_score(val_true, val_preds)
        
#         print(f'\nTraining Loss: {np.mean(train_losses):.4f}')
#         print(f'Training Accuracy: {train_accuracy:.4f}')
#         print(f'Validation Loss: {np.mean(val_losses):.4f}')
#         print(f'Validation Accuracy: {val_accuracy:.4f}')
        
#         if val_accuracy > best_accuracy:
#             best_accuracy = val_accuracy
#             print("\nSaving best model...")
#             torch.save(model.state_dict(), 'best_bert_model.pt')
            
#         print('\nClassification Report:')
#         print(classification_report(val_true, val_preds))

#     return model, tokenizer




def train_bert_model(data, epochs=3, batch_size=32, learning_rate=2e-5):
    """
    Train a BERT model for sentiment analysis with improved error handling
    """
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=2
    ).to(device)

    # Prepare data
    X = data['lemmatized_content'].values
    y = data['target'].values
    
    # Ensure target values are binary (0 or 1)
    y = np.where(y > 1, 1, y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # Create datasets
    train_dataset = TwitterDataset(X_train, y_train, tokenizer)
    val_dataset = TwitterDataset(X_val, y_val, tokenizer)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize optimizer with weight decay
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

    # Training loop with mixed precision
    scaler = torch.cuda.amp.GradScaler()
    best_accuracy = 0
    
    for epoch in range(epochs):
        print(f'\nEpoch {epoch + 1}/{epochs}')
        
        # Training phase
        model.train()
        train_losses = []
        train_preds = []
        train_true = []

        for batch in tqdm(train_loader, desc='Training'):
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            # Mixed precision training
            with torch.cuda.amp.autocast():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=targets
                )
                loss = outputs.loss

            # Scaled gradient
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_losses.append(loss.item())
            
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            train_preds.extend(preds)
            train_true.extend(targets.cpu().numpy())

        train_accuracy = accuracy_score(train_true, train_preds)
        
        # Validation phase
        model.eval()
        val_losses = []
        val_preds = []
        val_true = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                targets = batch['targets'].to(device)

                with torch.cuda.amp.autocast():
                    outputs = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=targets
                    )

                val_losses.append(outputs.loss.item())
                
                preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                val_preds.extend(preds)
                val_true.extend(targets.cpu().numpy())

        val_accuracy = accuracy_score(val_true, val_preds)
        
        print(f'\nTraining Loss: {np.mean(train_losses):.4f}')
        print(f'Training Accuracy: {train_accuracy:.4f}')
        print(f'Validation Loss: {np.mean(val_losses):.4f}')
        print(f'Validation Accuracy: {val_accuracy:.4f}')
        
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            print("\nSaving best model...")
            torch.save(model.state_dict(), 'best_bert_model.pt')
            
        print('\nClassification Report:')
        print(classification_report(val_true, val_preds))

    return model, tokenizer

In [22]:
def predict_sentiment(text, model, tokenizer, device=None):
    """
    Predict sentiment for a single text input with CUDA optimization
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        if device.type == 'cuda':
            with torch.cuda.amp.autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        else:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    
    return "Positive" if prediction == 1 else "Negative"

In [23]:
data = data.dropna()

In [24]:
# checking the distribution of target col

data['target'].value_counts()

target
1    25000
0    25000
Name: count, dtype: int64

In [25]:
# Ensure data is clean and preprocessed correctly
print(data['lemmatized_content'].head())
print(data['target'].unique())
print(data['target'].value_counts())

0    one reviewer mentioned watching episode youll ...
1    wonderful little production filming technique ...
2    thought wonderful way spend time hot summer we...
3    basically there family little boy jake think t...
4    petter matteis love time money visually stunni...
Name: lemmatized_content, dtype: object
[1 0]
target
1    25000
0    25000
Name: count, dtype: int64


In [26]:
model, tokenizer = train_bert_model(data)

Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\nikhi\AppData\Roaming\Python\Python312\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\2109533444.py:159: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



Epoch 1/3


Training:   0%|          | 0/1250 [00:00<?, ?it/s]C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\2109533444.py:179: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation:   0%|          | 0/313 [00:00<?, ?it/s]C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\2109533444.py:212: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation: 100%|██████████| 313/313 [00:39<00:00,  7.99it/s]



Training Loss: 0.3212
Training Accuracy: 0.8584
Validation Loss: 0.2551
Validation Accuracy: 0.8910

Saving best model...

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.86      0.89      5000
           1       0.87      0.92      0.89      5000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000


Epoch 2/3


Training:   0%|          | 0/1250 [00:00<?, ?it/s]C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\2109533444.py:179: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation:   0%|          | 0/313 [00:00<?, ?it/s]C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\2109533444.py:212: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation: 100%|██████████| 313/313 [00:39<00:00,  7.91it/s]



Training Loss: 0.1994
Training Accuracy: 0.9223
Validation Loss: 0.2521
Validation Accuracy: 0.8971

Saving best model...

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.91      0.90      5000
           1       0.91      0.88      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000


Epoch 3/3


Training:   0%|          | 0/1250 [00:00<?, ?it/s]C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\2109533444.py:179: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation:   0%|          | 0/313 [00:00<?, ?it/s]C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\2109533444.py:212: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation: 100%|██████████| 313/313 [00:39<00:00,  7.99it/s]


Training Loss: 0.1121
Training Accuracy: 0.9592
Validation Loss: 0.3180
Validation Accuracy: 0.8948

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      5000
           1       0.89      0.91      0.90      5000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [27]:
test_text = "This is a wonderful day! I'm really happy!"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(f"\nTest text: {test_text}")
print(f"Predicted sentiment: {sentiment}")


Test text: This is a wonderful day! I'm really happy!
Predicted sentiment: Positive


C:\Users\nikhi\AppData\Local\Temp\ipykernel_47484\3602477452.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [28]:
torch.cuda.is_available()


True

In [29]:
torch.cuda.device_count()

1

In [30]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3050 Laptop GPU'

# Sentiment Analysis Model Performance Report

## Model Overview
- **Model Type**: BERT-based Sequence Classification
- **Dataset**: IMDB Movie Reviews
- **Preprocessing**: Advanced text preprocessing with lemmatization and stopword removal

## Performance Metrics

| Metric | Value |
|--------|-------|
| **Overall Accuracy** | 89.48% |
| **Precision (Negative)** | 0.90 |
| **Recall (Negative)** | 0.88 |
| **F1-Score (Negative)** | 0.89 |
| **Precision (Positive)** | 0.89 |
| **Recall (Positive)** | 0.91 |
| **F1-Score (Positive)** | 0.90 |

## Training Progression

| Epoch | Training Loss | Training Accuracy | Validation Loss | Validation Accuracy |
|-------|--------------|-------------------|----------------|---------------------|
| 1 | 0.3212 | 85.84% | 0.2551 | 89.10% |
| 2 | 0.1994 | 92.23% | 0.2521 | 89.71% |
| 3 | 0.1121 | 95.92% | 0.3180 | 89.48% |

## Key Findings
1. Consistent high performance across epochs
2. Slight overfitting observed in later epochs
3. Balanced performance between positive and negative sentiment classification

## Model Configuration
- **Tokenizer**: BERT Base Uncased
- **Optimizer**: AdamW with weight decay
- **Learning Rate**: 2e-5
- **Batch Size**: 32
- **Training Epochs**: 3

## Preprocessing Techniques
- Lemmatization
- Stopword removal
- URL and special character removal
- Lowercasing

## Challenges and Limitations
- Potential domain-specific bias
- Limited to binary sentiment classification
- Performance may vary with different datasets

## Recommendations
1. Experiment with learning rate scheduling
2. Try longer training with early stopping
3. Explore domain-specific pre-training
4. Implement cross-validation for robust evaluation

## Conclusion
The BERT-based sentiment analysis model demonstrates strong performance with 89.48% accuracy, showing effective sentiment classification capabilities across movie reviews.